In [1]:
import pandas as pd
import json

def process_casualty_bins(file_path, output_json):
    cols = ['gname', 'nkill', 'nwound']
    df = pd.read_csv(file_path, encoding='ISO-8859-1', usecols=cols)
    df['nkill'] = df['nkill'].fillna(0)
    df['nwound'] = df['nwound'].fillna(0)

    group_mapping = {
        'ISIL': 'Islamic State of Iraq and the Levant (ISIL)',
        'taliban': 'Taliban',
        'SL': 'Shining Path (SL)'
    }

    bin_order = ["0", "1", "2-10", "11-100", "100+"]

    def assign_bin(val):
        if val == 0: return "0"
        if val == 1: return "1"
        if 2 <= val <= 10: return "2-10"
        if 11 <= val <= 100: return "11-100"
        if val > 100: return "100+"
        return "Unknown"

    output_data = {}

    for key, gtd_name in group_mapping.items():
        group_df = df[df['gname'] == gtd_name].copy()
        if group_df.empty:
            output_data[key] = []
            continue
        group_df['kill_bin'] = group_df['nkill'].apply(assign_bin)
        group_df['wound_bin'] = group_df['nwound'].apply(assign_bin)
        kill_counts = group_df['kill_bin'].value_counts()
        wound_counts = group_df['wound_bin'].value_counts()
        result = []
        for bin_label in bin_order:
            result.append({
                "bin": bin_label,
                "killed_count": int(kill_counts.get(bin_label, 0)),
                "wounded_count": int(wound_counts.get(bin_label, 0))
            })
        output_data[key] = result

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2)

process_casualty_bins('terrorism_dataset.csv', 'groups_casualty_bins.json')